<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/Transformer_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer のファインチューニング

### 必要なライブラリのインストール

In [1]:
!pip install transformers accelerate sentencepiece

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


### 言語モデルの準備

In [3]:
import transformers
import torch

# モデルとトークナイザの準備
model = transformers.AutoModelForCausalLM.from_pretrained(
    "cyberagent/open-calm-large",
    torch_dtype=torch.bfloat16
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "cyberagent/open-calm-large"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.78G [00:00<?, ?B/s]

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/326 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

### チューニング前のモデルによる生成

In [4]:
prompt = "休日の朝は"

input = tokenizer(
    prompt,
    return_tensors="pt"
)

outputs = model.generate(
    **input,
    max_length=64,
    do_sample=True,
    num_return_sequences=5
)

for i in range(len(outputs)):
    print("%d:" % (i+1))
    output = tokenizer.decode(outputs[i], skip_special_tokens=True)
    print(output)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


1:
休日の朝は、ゆっくり起きて、
今日は、ちょっと一息  午後ゆっくりコーヒーはいかがですか。
そして、今日のおやつは何かな・・・。
今日のおやつは、
【焼きたてパン あんぱん あんバタークリーム】です。
あんバタークリームを
生地にサンドしたパンの

2:
休日の朝は朝5時から7時まで働いて、そのあと7時まで残業して、帰りにスーパーで惣菜を買っておいて夕飯にして、そしてまた仕事をして、そしてまた夜7時まで残業って感じで家に帰ってまた仕事。もう嫌でイヤで仕方ないですよ」と打ち明けた。これを見たファンも「おぉ
3:
休日の朝はカフェでモーニング。
カフェ好きだけどモーニングって今まであんまり行ってなかったな。
そんなときに、
以前働いていた友人が新しいお店を開いたと言うので、
ランチでランチプレートを頼んでみて飲んでみました。
モーニングは、
6:30か7:30まで
モーニングはドリンクかトースト&
4:
休日の朝は、
早起きして、お香で気分をリセット。
一日のはじまりを気持ち良く迎えるためにも、
朝のお香習慣がおすすめです!
今日は私が香りを生活に取り入れている、
お香で香りを楽しむ、香りの楽しみ方です。
アロマテラピーやアロマタッチケア、
アロマトリートメント
5:
休日の朝は、愛犬とスキンシップしながらモーニングコーヒー。そしてゆっくり過ごす時間を大切にしています!
「自然豊かな軽井沢だからこそ、自然と共に人生を楽しむ!」をコンセプトに活動するネイチャークレンズコミュニティの代表。
ネイチャーアクネケアは、大自然の風土を生かしたオーガニックスキンケア。心と体・自然の健康を大切に考える方に、


### 学習データの用意
参考：HuggingFace Datasets の使い方｜npaka

In [5]:
import datasets

# データセットを用意
datadic = datasets.load_dataset("federerjiang/dialect.osaka")
dataset = datadic['train']

if len(dataset) > 1000: dataset = dataset.take(1000)  # データ量を減らす

README.md:   0%|          | 0.00/357 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/354M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1300 [00:00<?, ? examples/s]

### データセットの確認

In [6]:
# データセットを確認
print(dataset)

# pandas 形式に変換
dataset.set_format(type="pandas")
print(dataset[:])

Dataset({
    features: ['sentence', 'audio'],
    num_rows: 1000
})
                                      sentence  \
0    できるだけスマートフォンひとつで身の回りのことみんな片付けようとしてるみたいやで．   
1                             最近，下宿し始めたからちゃうか．   
2                               ほなあんたの紹介文いらんわ．   
3                     日本におるときやったら絶対考えられへんことやわ．   
4               てゆうか，インターネットから離れられへん生活が続いてんねん．   
..                                         ...   
995        蓬莱は豚まんが有名やけど、ほんまは海老焼売もめちゃめちゃ美味しいねん。   
996                        これやったら冷めてしもてもイケるしな。   
997                今日はちょっと贅沢してヘレのステーキでも食べに行こか。   
998                  たこ昌が有名やけど、うちは絶対くくるお勧めするわ。   
999                松葉の串カツ、テイクアウト出来るから、お土産にどない？   

                                                 audio  
0    {'path': 'osaka1300_0001.wav', 'array': [-3.46...  
1    {'path': 'osaka1300_0002.wav', 'array': [7.677...  
2    {'path': 'osaka1300_0003.wav', 'array': [8.283...  
3    {'path': 'osaka1300_0004.wav', 'array': [-1.88...  
4    {'path': 'osaka1300_0005.wav', 'array': 

## チューニングの実行

In [7]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 18.6 MB/s eta 0:00:00


In [11]:
# コレイタを準備
class CustomCollator(transformers.DataCollatorForLanguageModeling):
    def __call__(self, features):
        # トークン化
        batch = self.tokenizer(
            [feature['sentence'] for feature in features],
            padding=True, truncation=True,
            return_tensors="pt",
        )
        # ラベルの生成
        labels = batch['input_ids'].clone()
        labels[labels == self.tokenizer.pad_token_id] = -100
        batch['labels'] = labels
        return batch

# トレイナの準備
trainer = transformers.Trainer(
    model=model,
    data_collator=CustomCollator(  # 準備したコレイタを使う
        tokenizer,
        mlm=False
    ),
    args=transformers.TrainingArguments(
        output_dir="./output",
        num_train_epochs=5,
        per_device_eval_batch_size=1,
        remove_unused_columns=False  # データセットの列がコレイタに渡されるようにする
    ),
    train_dataset=dataset  # 加工しないデータセット
)


In [ ]:
# トレーニングする
trainer.train()

# 保存する
trainer.save_model("./trained_model")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

### チューニングしたモデルによる生成

In [9]:
# モデルとトークナイザの準備
model = transformers.AutoModelForCausalLM.from_pretrained(
    "./trained_model"
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "cyberagent/open-calm-large"
)

prompt = "休日の朝は"

input = tokenizer(
    prompt,
    return_tensors="pt"
)

outputs = model.generate(
    **input,
    max_length=64,
    do_sample=True,
    num_return_sequences=5
)

for i in range(len(outputs)):
    print("%d:" % (i+1))
    output = tokenizer.decode(outputs[i], skip_special_tokens=True)
    print(output)

OSError: Incorrect path_or_model_id: './trained_model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.